In [17]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [18]:
class_names = [
    "antilop", "porsuk", "yarasa", "ayı", "arı", "böcek", "bizon", "yaban domuzu",
    "kelebek", "kedi", "tırtıl", "şempanze", "hamam böceği", "inek", "çakal", "yengeç", 
    "karga", "geyik", "köpek", "yunus", "eşek", "helikopter böceği", "ördek", "kartal",
    "fil", "flamingo", "sinek", "tilki", "keçi", "japon balığı", "kaz", "goril", "çekirge",
    "hamster", "yaban tavşanı", "kirpi", "su aygırı", "kasklı guguk kuşu", "at", "sinek kuşu",
    "sırtlan", "denizanası", "kanguru", "koala", "uğur böceği", "leopar", "aslan", "kertenkele",
    "ıstakoz", "sivrisinek", "güve", "fare", "ahtapot", "okapi", "orangutan", "su samuru", "baykuş",
    "öküz", "istiridye", "panda", "papağan", "pelikan", "penguen", "domuz", "güvercin", "dikenli kirpi",
    "keseli sıçan", "rakun", "sıçan", "ren geyiği", "gergedan", "kıyı kuşu", "denizatı", "fok",
    "köpekbalığı", "koyun", "yılan", "serçe", "mürekkep balığı", "sincap", "deniz yıldızı", "kuğu",
    "kaplan", "hindi", "kaplumbağa", "balina", "kurt", "vombat", "ağaçkakan", "zebra"
]

test_dir = "C:\\Users\\Alperen Arda\\OneDrive\\Desktop\\YAZLAB\\YAZLAB_3.2\\3"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [19]:
from transformers import AutoConfig, AutoModelForImageClassification
import torch

config = AutoConfig.from_pretrained("microsoft/swinv2-tiny-patch4-window16-256")
config.num_labels = len(class_names)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForImageClassification.from_config(config)
model.load_state_dict(torch.load("swin/swin_model.pt", map_location=device))

<All keys matched successfully>

In [20]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(pixel_values=images).logits
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print("\n--- Test Set Classification Metrics ---")
print(f"Accuracy : %{accuracy * 100:.2f}")
print(f"Precision: %{precision * 100:.2f}")
print(f"Recall   : %{recall * 100:.2f}")
print(f"F1-Score : %{f1 * 100:.2f}")

print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(all_labels, all_preds)
print(f"Doğruluk (Accuracy): %{accuracy * 100:.2f}")